# <center> Feature engineering 

In [1]:
%store -r __ipy
%store -r __da

In [2]:
__ipy

Helper ipython script loaded


In [3]:
__da

Basic Data Analysis tools was loaded


In [ ]:
class TSFreshTransformer(BaseEstimator, TransformerMixin):

    def __init__(self, column_id, column_sort, column_value, extraction_settings):
        self.column_id = column_id
        self.column_sort = column_sort
        self.column_value = column_value
        self.extraction_settings = extraction_settings

    def fit(self, train_x, train_y=None, **fit_params):
        return self

    def transform(self, X_train, y_train=None, **fit_params):
        X_features = extract_features(
            X_train,
            column_id=self.column_id,
            column_sort=self.column_sort,
            column_value=self.column_value,
            default_fc_parameters=self.extraction_settings,
            disable_progressbar=True)

        impute(X_features)
        return X_features

    def get_params(self, **kwargs):
        return {'column_id': self.column_id,
                'column_sort': self.column_sort,
                'column_value': self.column_value,
                'extraction_settings': self.extraction_settings}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def fit_transform(self, X, y=None, **fit_params):
        return self.fit(X).transform(X)

In [ ]:
def generate_ts_features(df, X_train, X_test, it, month_threshold):
    def merge(df, features, it):
        return df.merge(features, how='left', left_on='key', right_on='id', right_index=False, suffixes=('', f'_{str(it)}'))
    
    print("item_cnt_day")
    features = TSFreshTransformer('key', 'date_block_num', 'item_cnt_day', EfficientFCParameters())\
        .fit_transform(df, axis=0)

    X_train = merge(X_train, features, it)
    X_test = merge(X_test, features, it)
    
    return X_train, X_test    

In [ ]:
def generate_ts(X_train_val, y_train_val, X_test):
    
    assert X_train_val.shape[0] > X_test.shape[0]
    assert X_train_val.shape[0] == y_train_val.shape[0]
    
    X_train_val_func = X_train_val.copy()
    X_test_func = X_test.copy()

    X = pd.concat([X_train_val.reset_index(), y_train_val.reset_index()], axis=1)

    for i, month in tqdm(enumerate([12, 24])):
        X_train_val_func, X_test_func = generate_ts_features(X, X_train_val_func, X_test_func, month, month)
        
    X_train_val_func = X_train_val_func.fillna(0)
    X_test_func = X_test_func.fillna(0)
  
    return X_train_val_func, X_test_func